## _Import_

In [3]:
import tensorflow as tf
import numpy as np
import pickle
import random
import time
import os

from tensorflow.keras import layers

In [4]:
tf.__version__

'2.2.0'

## _Dummy Data_

In [ ]:
data = [np.ones([1,4, 4,1], dtype = int) for x in range(100)] 

In [ ]:
data[5].shape

In [ ]:
data[5]

## _The Generator_

In [2]:
def generator_model(n):
    model = tf.keras.Sequential()
    model.add(layers.Dense(n*n*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((n, n, 256)))
    assert model.output_shape == (None, n, n, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, n, n, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 2*n, 2*n, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 2*n, 2*n, 1)
    return model

In [3]:
def discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[24, 24, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [144]:
generator = generator_model(2)

In [145]:
generator.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 200)               20000     
_________________________________________________________________
batch_normalization_16 (Batc (None, 200)               800       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 200)               0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 2, 2, 25)          0         
_________________________________________________________________
conv2d_transpose_15 (Conv2DT (None, 2, 2, 25)          15625     
_________________________________________________________________
batch_normalization_17 (Batc (None, 2, 2, 25)          100       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 2, 2, 25)        

In [ ]:
discriminator = discriminator_model()

In [35]:
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 2, 2, 64)          1664      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 2, 2, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 128)         204928    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)              

### _Generator Test_

In [112]:
noise = tf.random.normal([1, 100])
generated_matrix = generator(noise, training=False)

InvalidArgumentError: Input to reshape is a tensor with 14400 values, but the requested shape has 21600 [Op:Reshape]

In [ ]:
generated_matrix

### _Discriminator Test_

In [ ]:
test = tf.reshape(data[0], [1,4,4,1])
test

In [ ]:
discriminator(test)

In [ ]:
discriminator(generated_matrix)

In [ ]:
decision = discriminator(generated_matrix)
print (decision)

## _Loss and Optimizer_

In [4]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def D_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
  
def G_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [5]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## _Training Functions_

In [ ]:
noise_dim = 100
num_of_generated_examples = 16

seed = tf.random.normal([num_of_generated_examples, noise_dim])

In [ ]:
generator = generator_model(12)
discriminator = discriminator_model()

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

#@tf.function
def train_step(adj_matrix):
    noise = tf.random.normal([100, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_matrix = generator(noise, training=True)
        
        real_output = discriminator(adj_matrix, training=True)
        fake_output = discriminator(generated_matrix, training=True)

        gen_loss = G_loss(fake_output)
        disc_loss = D_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

## _Train_

In [169]:
def train_GAN(dataset, epochs, n):
    for epoch in range(epochs):
        start = time.time()
        
        for batch in dataset:

            train_step(batch.reshape(1, n, n, 1))
        # Save the model every 15 epochs
        if (epoch + 1) % 15 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [170]:
#DWave Data
file = open('dWave.pickle', 'rb')
data = pickle.load(file)
#data[0].reshape(1,24,24,1)

In [172]:
train_GAN(data, 100, 24)

Time for epoch 1 is 5.06377387046814 sec
Time for epoch 2 is 4.9691245555877686 sec
Time for epoch 3 is 5.0002031326293945 sec
Time for epoch 4 is 5.008155345916748 sec
Time for epoch 5 is 4.978732347488403 sec
Time for epoch 6 is 5.021666049957275 sec
Time for epoch 7 is 4.975025415420532 sec
Time for epoch 8 is 5.038977384567261 sec
Time for epoch 9 is 4.9627134799957275 sec
Time for epoch 10 is 4.9984290599823 sec
Time for epoch 11 is 4.990645408630371 sec
Time for epoch 12 is 5.0082480907440186 sec
Time for epoch 13 is 5.006568193435669 sec
Time for epoch 14 is 4.9933905601501465 sec
Time for epoch 15 is 5.326875448226929 sec
Time for epoch 16 is 5.078976392745972 sec
Time for epoch 17 is 4.997675657272339 sec
Time for epoch 18 is 5.0037689208984375 sec
Time for epoch 19 is 4.982633590698242 sec
Time for epoch 20 is 4.969334602355957 sec
Time for epoch 21 is 4.96699070930481 sec
Time for epoch 22 is 4.96765398979187 sec
Time for epoch 23 is 5.006393194198608 sec
Time for epoch 24 i

In [ ]:
noise = tf.random.normal([1, 100])
gen_2 = generator(noise, training=False)

In [ ]:
gen_2

## _Saving Model_

In [173]:
generator.save('DwaveModel2/')

INFO:tensorflow:Assets written to: DwaveModel2/assets


In [174]:
enigma = tf.keras.models.load_model('DwaveModel2/', compile=False)

In [175]:
enigma

In [176]:
enigma.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 36864)             3686400   
_________________________________________________________________
batch_normalization_25 (Batc (None, 36864)             147456    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 36864)             0         
_________________________________________________________________
reshape_9 (Reshape)          (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_transpose_24 (Conv2DT (None, 12, 12, 128)       819200    
_________________________________________________________________
batch_normalization_26 (Batc (None, 12, 12, 128)       512       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 12, 12, 128)     

In [ ]:
noise = tf.random.normal([1, 100])

In [ ]:
noise

In [ ]:
enigma(noise, training=False)

## _Adj Matrix_

In [11]:
def random_adjacency_matrix(n):
    """
    creates an nxn symmetric adjacency matrix with 0s along the diagonal
    used to represent an undirected graph with n nodes
    :param n: dimension
    :return: nxn numpy array
    """
    matrix = [[random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return np.array(matrix)

In [12]:
random_adjacency_matrix(4)

array([[0, 1, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [0, 0, 1, 0]])

In [13]:
data = [random_adjacency_matrix(4).reshape([1,4,4,1]) for x in range(500)] 

In [ ]:
test = data[0]

In [ ]:
test

In [ ]:
test.reshape([4,4])

In [ ]:
data[0]

## _Train Enigma 2 with better Dummy Data_

In [38]:
data_2 = [random_adjacency_matrix(4).reshape([1,4,4,1]) for x in range(100)] 

In [39]:
train_GAN(data_2, 100)

Time for epoch 1 is 2.7683157920837402 sec
Time for epoch 2 is 2.6759774684906006 sec
Time for epoch 3 is 2.647869110107422 sec
Time for epoch 4 is 2.832427978515625 sec
Time for epoch 5 is 2.7549357414245605 sec
Time for epoch 6 is 2.7482006549835205 sec
Time for epoch 7 is 2.8244383335113525 sec
Time for epoch 8 is 2.900914192199707 sec
Time for epoch 9 is 2.6654133796691895 sec
Time for epoch 10 is 2.6945090293884277 sec
Time for epoch 11 is 2.712099552154541 sec
Time for epoch 12 is 2.6996915340423584 sec
Time for epoch 13 is 2.6906418800354004 sec
Time for epoch 14 is 2.680401563644409 sec
Time for epoch 15 is 2.7673134803771973 sec
Time for epoch 16 is 2.7330236434936523 sec
Time for epoch 17 is 2.700709342956543 sec
Time for epoch 18 is 2.707001209259033 sec
Time for epoch 19 is 2.799954414367676 sec
Time for epoch 20 is 2.703674554824829 sec
Time for epoch 21 is 2.659414052963257 sec
Time for epoch 22 is 2.6714985370635986 sec
Time for epoch 23 is 2.673480272293091 sec
Time for

In [40]:
generator.save('Model/')

INFO:tensorflow:Assets written to: Model/assets


## _Load Model_

In [2]:
enigma = tf.keras.models.load_model('DwaveModel2/', compile=False)

NameError: name 'tf' is not defined

In [178]:
enigma.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 36864)             3686400   
_________________________________________________________________
batch_normalization_25 (Batc (None, 36864)             147456    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 36864)             0         
_________________________________________________________________
reshape_9 (Reshape)          (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_transpose_24 (Conv2DT (None, 12, 12, 128)       819200    
_________________________________________________________________
batch_normalization_26 (Batc (None, 12, 12, 128)       512       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 12, 12, 128)     

In [179]:
noise = tf.random.normal([1, 100])
cat = enigma(noise, training=False)

In [181]:
test = tf.reshape(cat, [24,24])

In [183]:
bo = tf.math.greater(test, 0.5)

In [184]:
tf.cast(bo, tf.int32)

<tf.Tensor: shape=(24, 24), dtype=int32, numpy=
array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [185]:
output = []
for i in range(50):
    noise = tf.random.normal([1, 100])
    cat = enigma(noise, training=False)
    kitty = tf.reshape(cat, [24,24])
    fluff = tf.math.greater(kitty, 0.5)
    puff = tf.cast(fluff, tf.int32)
    print(puff)
    output.append(puff)

tf.Tensor(
[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 

In [141]:
output

[<tf.Tensor: shape=(24, 24), dtype=int32, numpy=
 array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,